# LLM - RAG (Large Language Models - Retrievel-Augmented Generation)

# Overview
This demo showcases how to use the GPT-3 model in a Retrieval-Augmented Generation (RAG) setup. \
The model analyzes a natural language query, retrieves relevant information from a knowledge base, and provides an answer based on the retrieved information.



In [10]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')

### Start with a natural question

In [11]:
input = "Which presendential candidate is likely to win in 2024 election? Explain the reasons."

### Step 1: GPT3: What do I need to to answer this question?

In [12]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [13]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

It is too early to accurately predict the outcome of the 2024 presidential election as it is still several years away. However, I can research current polls, political trends, and potential candidates to make an educated guess. 
Action: Research 
Action Input: Current polling data, political news and trends, potential candidates.


### Step 2: Search web for more details 
Before continuing please create a Bing Search Service in your subscription [Create Bing Search Resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource). \
This sample makes a call to the Bing Web Search API with a query and returns relevant web search.

In [37]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.getenv('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis"}
response = requests.get(url, headers=headers, params=params)

In [38]:
context = ''
print(response.json())
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

{'_type': 'SearchResponse', 'queryContext': {'originalQuery': '2024 presidential election candidates and current political analysis'}, 'webPages': {'webSearchUrl': 'https://www.bing.com/search?q=2024+presidential+election+candidates+and+current+political+analysis', 'totalEstimatedMatches': 767000, 'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0', 'name': 'Latest Election 2024 Polls: Biden vs. Trump - The New York Times', 'url': 'https://www.nytimes.com/interactive/2024/us/elections/polls-president.html', 'datePublished': '2024-06-24T00:00:00.0000000', 'datePublishedFreshnessText': '3 days ago', 'isFamilyFriendly': True, 'displayUrl': 'https://www.nytimes.com/interactive/2024/us/elections/polls-president.html', 'snippet': 'Our polling averages track the latest trends in the presidential race, using data from national and battleground state polls. ... On Politics Newsletter. The latest news and analysis on the 2024 ...', 'deepLinks': [{'name': 'Who Are the 2024 Presid

### Collected information for further processing

In [39]:
print(context)

Our polling averages track the latest trends in the presidential race, using data from national and battleground state polls. ... On Politics Newsletter. The latest news and analysis on the 2024 ...
Joe Biden has about a 1 in 3 chance. 2020 results. 306 306 232 232. Likely range. Median. Our model is updated every day and combines state and national polls with economic indicators to predict ...
The latest political polls and polling averages from FiveThirtyEight. UPDATED Jun. 26, 2024, at 9:02 PM. ... Updating average for each candidate in 2024 presidential polls, accounting for each poll's recency, sample size, methodology and house effects. ... general election, 2024. AVG. June 22-24. 878: LV: June 22-24. 878 LV. Leger. The New York ...
The presidential election is 132 days away. Here’s our guide to the run-up to Election Day. ... On Politics Newsletter. The latest news and analysis on the 2024 elections, sent to your inbox.
2024 election guide: Presidential candidates, polls, primar

In [40]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [41]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

According to the context provided, as of now, the 2024 presidential race is considered a pure toss-up, with President Joe Biden slightly favored to win in 501 out of 1,000 of their model's simulations. However, former President Donald Trump wins in 495 of their simulations, indicating a close race. The latest political polls and polling averages from FiveThirtyEight show updated averages for each candidate in the 2024 presidential polls, accounting for each poll's recency, sample size, methodology, and house effects. It is important to note that polling averages can shift even if no new polls have been added to the page. So, it is unclear which presidential candidate is likely to win in the 2024 election as of now, and the race appears to be very close.


### GPT3: I now have final answer to the question. 

#### Use cases to explore and think about with the customer
1. **Knowledge Management** \
Improve access to corporate knowledge by retrieving and presenting information from internal documents, manuals, and policies.

2. **Business Intelligence and Analytics** \
Support market analysts by retrieving relevant market data, reports, and insights from various sources.

3. **Legal and Compliance** \
Aid legal professionals by retrieving relevant case laws, statutes, and legal precedents based on queries.